# Cyclic Voltammograms

### Imports

In [1]:
import ase
from ase.db import connect
import numpy as np

### Create surface

The five metals used are (in alphabetical order): Ag, Au, Cu, Pd, Pt. A 100x100 surface in three layers with an even metal-distribution is created in a simple NumPy array. 

In [2]:
metals = ['Ag', 'Au', 'Cu', 'Pd', 'Pt']

In [17]:
# Make a 100x100x3 surface with an even distribution of the five metals

dim_x, dim_y, dim_z = 100, 100, 3 #Specify dimensions

surface_list = np.array([int(dim_x*dim_y*dim_z/len(metals))*[metals[metal_number]] for metal_number in range(len(metals))]).flatten() #Jack had a way shorter way of doing this, but I think it was random drawing instead of ensuring a perfectly even split
np.random.shuffle(surface_list) #Shuffle list
surface = np.reshape(surface_list, (dim_x, dim_y, dim_z)) #Reshape list to the 

In [12]:
surface.shape

(100, 100, 3)

### Extract relevant neighbours from a site

#### On top

In [32]:
# For each top position find the relevant positions and save in the sites_vector
site_x = 1
site_y = 1

In [21]:
# Find the "site vector" for a specific site, according to the Jack article
# Perhaps at one point I'll add the actual metal as the first input

def on_top_site_vector(surface, site_x, site_y):
    
    #sites_vector = [0 for n in range(15)] Maybe just smash top6, mid3 and bot3 together

    top6 = [surface[site_x % 100, (site_y-1) % 100, 0], surface[site_x % 100, (site_y+1) % 100, 0], surface[(site_x-1) % 100, site_y % 100, 0], surface[(site_x+1) % 100, site_y % 100, 0], surface[(site_x-1) % 100, (site_y+1) % 100, 0], surface[(site_x+1) % 100, (site_y-1) % 100, 0]]
    top6_count = [top6.count(metals[n]) for n in range(len(metals))]
    
    mid3 = [surface[(site_x-1) % 100, (site_y-1) % 100,1], surface[site_x % 100, (site_y-1) % 100,1], surface[(site_x-1) % 100, site_y % 100,1]]
    mid3_count = [mid3.count(metals[n]) for n in range(len(metals))]
    
    bot3 = [surface[(site_x-1) % 100, (site_y-1) % 100, 2], surface[(site_x-1) % 100, (site_y+1) % 100, 2], surface[(site_x+1) % 100, (site_y-1) % 100, 2]]
    bot3_count = [bot3.count(metals[n]) for n in range(len(metals))]
    
    return top6_count + mid3_count + bot3_count

# I need a function to take a site_vector and return an energy from the linear model
# The model depends on the metal, I guess that is a but funky. Why not use 1 model?

In [24]:
# Finding the site vector for all on-top sites on the 100x100 surface (Without border)

for site_x in range(100): #Looping through all on top sites
    for site_y in range(100):
        
        # Create vector for on-top site
        on_top_vec = on_top_site_vector(surface, site_x, site_y)
        
        # Create vector for hollow site / fcc site
        # not done yet - how many hollow sites are there?
        
        #print(on_top_vec) # Prints all site_vectors
        
        # CALCULATE ENERGY HERE! FOR ALL REACTIONS
        
        # which we actually need models for. I think we only have a model for OH eller O
        # When you get here, you need to make simple models for all three adsorbates

In [33]:
site_vec = on_top_site_vector(surface, site_x, site_y)
print(site_vec)

[2, 0, 1, 1, 2, 0, 1, 0, 2, 0, 1, 0, 1, 1, 0]


In [34]:
# Inspect the actual surface to check if the function pulls the right atoms
# Print the top layer of the surface
print("    Top layer  of the surface")
print(surface[0:6, 0:6, 0])
print("Site: ", surface[site_x, site_y, 0])

print("\n    Mid layer  of the surface")
print(surface[0:6, 0:6, 1])

print("\n    Bot layer  of the surface")
print(surface[0:6, 0:6, 2])


    Top layer  of the surface
[['Au' 'Pt' 'Cu' 'Pt' 'Au' 'Cu']
 ['Pt' 'Pd' 'Ag' 'Cu' 'Au' 'Au']
 ['Ag' 'Pd' 'Pd' 'Cu' 'Pt' 'Pd']
 ['Cu' 'Pd' 'Pd' 'Ag' 'Pt' 'Pt']
 ['Pd' 'Pt' 'Pd' 'Pt' 'Pt' 'Pd']
 ['Au' 'Cu' 'Pt' 'Au' 'Pt' 'Pt']]
Site:  Pd

    Mid layer  of the surface
[['Au' 'Pd' 'Ag' 'Pt' 'Au' 'Cu']
 ['Pd' 'Cu' 'Pd' 'Au' 'Pt' 'Ag']
 ['Au' 'Cu' 'Au' 'Ag' 'Ag' 'Pt']
 ['Pt' 'Pd' 'Ag' 'Au' 'Cu' 'Cu']
 ['Pd' 'Pd' 'Pd' 'Pd' 'Cu' 'Pd']
 ['Cu' 'Ag' 'Au' 'Cu' 'Pd' 'Ag']]

    Bot layer  of the surface
[['Ag' 'Pd' 'Cu' 'Pt' 'Cu' 'Pd']
 ['Pd' 'Pd' 'Ag' 'Au' 'Cu' 'Pt']
 ['Pd' 'Ag' 'Pt' 'Pt' 'Ag' 'Ag']
 ['Pd' 'Cu' 'Cu' 'Cu' 'Ag' 'Pt']
 ['Au' 'Pt' 'Pt' 'Au' 'Au' 'Au']
 ['Ag' 'Pt' 'Au' 'Pt' 'Pd' 'Cu']]


#### Hollow sites

In [35]:
three_metals_combinations = [] #List of possible combinations of the three
# Der skal være 35, ikke 125

for a in metals:
    for b in metals:
        for c in metals:
            three_metals_combinations.append(''.join(sorted([a, b, c])))
            
# Remove duplicates
three_metals_combinations = list(dict.fromkeys(three_metals_combinations))

# Let's encode it in a better way later

In [36]:
# Find the "site vector" for a specific site, according to the Jack article

def hollow_site_vector(surface, site_x, site_y):
    # First encode the 3 neighbours
    close_3s = [surface[(site_x+1) % 100, site_y, 0], surface[site_x, (site_y+1) % 100, 0], surface[(site_x+1) % 100, (site_y+1) % 100, 0]]
    close_3s = "".join(sorted(close_3s))
    idx = three_metals_combinations.index(close_3s)
    close_3s = np.zeros(35)
    close_3s[idx] = 1
    
    # Then the next neighbours (green)
    greens = [surface[(site_x+2) % 100, site_y, 0], surface[site_x, (site_y+2) % 100, 0], surface[site_x, site_y, 0]]
    greens_count = [greens.count(metals[n]) for n in range(len(metals))]
    
    # Then the next neighbours (brown)
    browns = [surface[(site_x) % 100, (site_y) % 100, 0], surface[(site_x) % 100, (site_y) % 100, 0], surface[(site_x) % 100, (site_y) % 100, 0], surface[(site_x) % 100, (site_y) % 100, 0], surface[(site_x) % 100, (site_y) % 100, 0], surface[(site_x) % 100, (site_y) % 100, 0]]
    # Kunne gøres smartede med list comprehension og to lister med +- zipped
    return None

hollow_site_vector(surface, site_x, site_y)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
